In [ ]:
import pandas as pd
import numpy as np
from IPython.display import Markdown
from plotly import graph_objects as go
import plotly.express as px


import tb_incubator.constants as const
from tb_incubator.constants import set_project_base_path
from tb_incubator.input import get_pop_death_data, get_death_rates
from tb_incubator.model import build_model
from tb_incubator.input import load_param_info
from tb_incubator.plotting import display_plot

pd.options.plotting.backend = "plotly"
project_paths = set_project_base_path("../tb_incubator/")

## Population data

We used estimates for the total population, deaths, and birth rate from the United Nations World Population Prospects [@unwpp2024].

In [ ]:
# Load age-stratified, population and death data
pop_death, description = get_pop_death_data()
target_pops = pop_death.groupby(level=[0]).sum()["population"]
Markdown(description)

## Model construction
This is arbitrary, but including some epidemiological transitions in the background.

In [ ]:
compartments = const.COMPARTMENTS
age_strata = const.AGE_STRATA
covid_effects = {
    'detection_reduction':True
}

In [ ]:
param_info = load_param_info()
fixed_params = param_info["value"]
params = {
    'base_diagnostic_capacity': 0.6759881242729308,
    'contact_rate': 63.36486942195536,
    'detection_reduction': 0.6006425343385974,
    'genexpert_sensitivity': 0.9215736965003359,
    'incidence_props_smear_positive_among_pulmonary': 0.590232769667581,
    'initial_notif_rate': 0.4649436995023598,
    'latest_notif_rate': 0.7915609033439653,
    'rr_infection_latent': 0.17725105090258983,
    'rr_infection_recovered': 0.5096458681008372,
    'screening_inflection_time': 2013.3492360665543,
    'screening_scaleup_shape': 0.08648392571375052,
    'smear_negative_death_rate': 0.025911961086601855,
    'smear_negative_self_recovery': 0.1399643939493157,
    'smear_positive_death_rate': 0.42933478679483505,
    'smear_positive_self_recovery': 0.24182351464224994,
    'time_to_screening_end_asymp': 1.7890398328562447
}

fixed_params.update(params)
model, desc = build_model(fixed_params, xpert_improvement=True, covid_effects=covid_effects)
Markdown(desc)

## Demographic results

In [ ]:
# Run and inspect results
file_prefix = "ID"
model.run(fixed_params)

df = model.get_derived_outputs_df()[[f"total_populationXage_{age}" for age in age_strata]]
fig = px.area(df, x=df.index, y=[f"total_populationXage_{age}" for age in age_strata])
fig.add_trace(go.Scatter(x=target_pops.index, y=target_pops, name="target", mode="markers", marker=dict(color="black", size=2.0)))
#display_plot(fig, f"{file_prefix}_comp_size", "svg")